In [1]:
from dataclasses import dataclass
from typing import List, Type
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim

In [2]:
@dataclass
class Input:
    name: str
    age: int


@dataclass
class InputBatch:
    
    def from_inputs(cls, inputs: List[Type[Input]]):
        return cls(inputs)


class NetworkBase(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    # def forward(self, batch):
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        # return OutputBatch.from_input_batch(batch)

    def train_step(self, batch: Type[InputBatch]):
        return self(batch)

    @torch.no_grad()
    def step(self, batch: Type[InputBatch]):
        return self(batch)

@dataclass
class OutputBatch:
    
    def from_input_batch(cls, input_batch: Type[InputBatch], logits):
        return cls(input_batch, logits)


@dataclass
class Output:
    
    @classmethod
    def from_output_batch(cls, output_batch):
        return [cls(output) for output in output_batch]


In [7]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [8]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # loop over the dataset multiple times

    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.cuda())
        loss = criterion(outputs, labels.cuda())
        loss.backward()
        optimizer.step()

        # print statistics
        print(f'\r[{epoch + 1}, {i + 1:5d}] loss: {loss.item():.3f}', end='')

print()
print('Finished Training')

[2,   782] loss: 2.126
Finished Training


In [ ]:
class Data(type):

    def __new__(cls, name, bases, class_dict):
        return super().__new__(cls, name, bases, class_dict)

In [ ]:
class MyMeta(type):
    def __new__(cls, name, bases, attr_dict):
        print('this is meta', cls.__name__, name)
        print(attr_dict.items())
        return super().__new__(cls, name, bases, attr_dict)

@dataclass
class MyClass(metaclass=MyMeta):
    name: str

@dataclass
class MySubclass(MyClass):
    age: int

myclass = MyClass('Khoa')
print(myclass, type(myclass))

mysubclass = MySubclass('Khoa', 29)
print(mysubclass, type(mysubclass))